In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from pyvis.network import Network
import pyvis
from community import community_louvain
import matplotlib.colors as mcolors

In [2]:
#carga candidatos y pactos
pactos = pd.read_csv('https://storage.googleapis.com/notas-blog-public/03%20-%20Red%20candidatos/nodes_cc.csv', encoding = 'latin1', usecols = ['id', 'pacto', 'partido'])
pactos['id'] = pactos['id'].str.lower()

# Número de candidatxs por pacto
pactos.groupby('pacto').size().sort_values(ascending = False)

pacto
VAMOS POR CHILE                                   117
LISTA DEL APRUEBO                                 111
APRUEBO DIGNIDAD                                  104
PARTIDO DE TRABAJADORES REVOLUCIONARIOS            28
PARTIDO ECOLOGISTA VERDE                           24
                                                 ... 
LISTA INDEPENDIENTE PARTO SOCIAL (D7)               1
LISTA DEL PUEBLO-RIOS INDEPENDIENTES (D24)          1
LA LISTA DEL PUEBLO DISTRITO 14 (D14)               1
INDEPENDIENTES POR LA NUEVA CONSTITUCION (D24)      1
VOCES CONSTITUYENTES (D12)                          1
Length: 71, dtype: int64

In [3]:
#re-agrupamos las listas
lista = ['VAMOS POR CHILE', 'LISTA DEL APRUEBO', 'APRUEBO DIGNIDAD', 'PARTIDO DE TRABAJADORES REVOLUCIONARIOS']
pactos.loc[~pactos.pacto.isin(lista), 'pacto'] = 'INDEPS Y OTROS'

In [4]:
#carga edgelist candidatos en Twitter
red_cc = pd.read_csv('https://storage.googleapis.com/notas-blog-public/03%20-%20Red%20candidatos/edgelist_cc.csv')
red_cc.user = red_cc.user.str.lower()
red_cc.screen_name = red_cc.screen_name.str.lower()

In [6]:
#diccionario de pactos por cuenta
pactos_dict = dict(zip(pactos.id, pactos.pacto))

In [7]:
#creamos la red con networkx
nx_graph = nx.from_pandas_edgelist(red_cc, source = 'user', target = 'screen_name')
print('Grafo con ' + str(nx_graph.number_of_nodes()) + ' nodos')

#identificamos las comunidades usando el algoritmo Louvain
coms = community_louvain.best_partition(nx_graph)
print('\nComunidades:')
print(set(list(coms.values())))

#Definimos colores
all_colors = ['red','grey','blue','green', '#9ACD32','grey','#DDA0DD']
n = len(set(list(coms.values())))
colors = list(all_colors)[:len(coms)]

#creamos el atributo color
node_names = list(nx_graph.node.keys())
for i, p in enumerate(list(coms.values())):
    nx_graph.node[node_names[i]]['color'] = colors[p]
colors_2 = nx.get_node_attributes(nx_graph, 'color').values()

#creamos el atributo title, el nombre de cada nodo
node_names = list(nx_graph.node.keys())
for i, p in enumerate(nx_graph.nodes()):
    nx_graph.node[node_names[i]]['title'] = p

#calculamos grado y lo asignamos a los pesos (size) de los nodos
#mantenemos los tamaños en el rango 10-60 para efectos de visualización
degree_dict = dict(nx_graph.degree())
for k, v in list(degree_dict.items()):
    if v > 80:
        degree_dict[k] = 60
    if v < 10:
        degree_dict[k] = 10
nx.set_node_attributes(nx_graph, degree_dict, "size")

Grafo con 589 nodos

Comunidades:
{0, 1, 2, 3, 4, 5, 6}


In [8]:
#Graficamos con pyvis, coloreado por COMUNIDAD
net = Network(height='800px', width='800px', directed=False, notebook=True, bgcolor='yellow', 
              font_color=False, layout=None)
net.from_nx(nx_graph)
net.force_atlas_2based(gravity=-50, central_gravity=0.01, spring_length=100, spring_strength=0.08, damping=0.4, overlap=0)
net.show('red_cc_COM.html')

In [9]:
#creamos el atributo 'pacto'
node_names = list(nx_graph.node.keys())
for nombre in node_names:
    if nombre in pactos_dict.keys():
        nx_graph.node[nombre]['pacto'] = pactos_dict[nombre]
    else:
        nx_graph.node[nombre]['pacto'] = 'INDEPS Y OTROS'
        
#Definimos colores por pacto
all_colors = ['#DDA0DD', '#9ACD32', 'red','blue','green','grey','#DDA0DD']
n = len(pactos.pacto.unique())
colors = list(all_colors)[:n]

pacto_color = dict(zip(list(pactos.pacto.unique()), colors))
pacto_color

{'LISTA DEL APRUEBO': '#DDA0DD',
 'INDEPS Y OTROS': '#9ACD32',
 'PARTIDO DE TRABAJADORES REVOLUCIONARIOS': 'red',
 'VAMOS POR CHILE': 'blue',
 'APRUEBO DIGNIDAD': 'green'}

In [10]:
#creamos el atributo color (para los pactos)
node_names = list(nx_graph.node.keys())
for nombre in node_names:
    nx_graph.node[nombre]['color'] = pacto_color[nx_graph.node[nombre]['pacto']]    

In [11]:
#Graficamos con pyvis, coloreado por PACTO
net = Network(height='600px', width='600px', directed=False, notebook=True, bgcolor='yellow', 
              font_color=False, layout=None)
net.from_nx(nx_graph)
net.force_atlas_2based(gravity=-50, central_gravity=0.01, spring_length=100, spring_strength=0.08, damping=0.4, overlap=0)

net.show('red_cc_PACTO.html')